In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt


# odszumianie sygnału

df = pd.read_csv('../../data/partially processed/resampled_data_no_outliers.csv')
df.drop(columns=['outlier_label'], inplace=True)
df['epoch (ms)'] = pd.to_datetime(df['epoch (ms)'])
df.set_index('epoch (ms)', inplace=True)

def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y



# calate set duration
for i in df["set"].unique():
    duration = df[df["set"] == i].index[-1] - df[df["set"] == i].index[0]
    df.loc[df["set"] == i, "duration"] = duration.seconds
    
duration_data = df.groupby("category")["duration"].mean()
    





In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11963 entries, 2019-01-11 15:08:05.250000 to 2019-01-20 17:33:27.750000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        11963 non-null  float64
 1   acc_y        11963 non-null  float64
 2   acc_z        11963 non-null  float64
 3   gyro_x       11963 non-null  float64
 4   gyro_y       11963 non-null  float64
 5   gyro_z       11963 non-null  float64
 6   participant  11963 non-null  object 
 7   label        11963 non-null  object 
 8   category     11963 non-null  object 
 9   set          11963 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 1.0+ MB


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


df.reset_index(drop=True, inplace=True)
# Prepare the data
X = df[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Initialize the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9289520426287744
Classification Report:
               precision    recall  f1-score   support

       bench       0.92      0.88      0.90      1769
        dead       0.92      0.91      0.91      1630
         ohp       0.88      0.91      0.90      1796
        rest       0.98      0.99      0.98      1167
         row       0.91      0.90      0.91      1489
       squat       0.98      1.00      0.99      1720

    accuracy                           0.93      9571
   macro avg       0.93      0.93      0.93      9571
weighted avg       0.93      0.93      0.93      9571

